In [80]:
import pandas as pd
from textblob import TextBlob 
import re
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = -1
tweets = pd.read_csv('tweets_premier_1819.csv')
tweets.head()

,match_round,home_team,away_team,team,tweet_text,tweet_id,tweet_username,tweet_date,tweet_retweets,tweet_favorites,tweet_mentions,tweet_hashtags,tweet_geo
0,38,Brighton,Manchester City,Brighton,Cannot decide who I love more between him and Calderón #bhafc,1126999402273562625,_GeorgeCarden,2019-05-10 23:56:10+00:00,0,2,NaN,#bhafc,NaN
1,38,Brighton,Manchester City,Brighton,Bruno embodies everything we would love our club to mean: ✅Dignity ✅Compassion ✅Family ✅Liberalism ✅Flair ✅Professionalism ✅Leadership We’re going to miss you and your bloody great beard on the pitch. #Bruno #bhafc,1126990216105988098,DrCols,2019-05-10 23:19:40+00:00,8,40,NaN,#Bruno #bhafc,NaN
2,38,Brighton,Manchester City,Brighton,"I might go clubbing in #brighton , Do you fancy going to west street? we can go out in our #bhafc tops",1126985277313044480,BillArcherBHAFC,2019-05-10 23:00:03+00:00,0,0,NaN,#brighton #bhafc,NaN
3,38,Brighton,Manchester City,Brighton,"To show my true faith to the masses, and proove my blue and white blood, perhaps I should go for Selhurst Park. (or SELLhurst) #bhafc #cpfc That would be epic",1126985028167188480,BillArcherBHAFC,2019-05-10 22:59:03+00:00,0,0,NaN,#bhafc #cpfc,NaN
4,38,Brighton,Manchester City,Brighton,Thank you El Capitan! #BHAFC #GraciasBruno #mfootballw #football,1126983891506597891,MyFootballWorl3,2019-05-10 22:54:32+00:00,0,0,NaN,#BHAFC #GraciasBruno #mfootballw #football,NaN


In [81]:
# remove duplicated tweets
tweets = tweets[tweets.tweet_favorites>1].drop_duplicates(['tweet_id']).reset_index(drop=True)

In [82]:
HASTAGS = {
    'Arsenal':  ['#afc','#coyg','#arsenal', '#arsenalfc','#gunners'],
    'Aston Villa': ['#avfc','#astonvilla', '#astonvillafc'],
    'Bournemouth': ['#afcb','#bournemouth'],
    'Brighton': ['#bhafc', '#brightonandhovealbion','#brightonfc'],
    'Burnley': ['#clarets','#burnley', '#burnleyfc'],
    'Cardiff': ['#cardiffcity','#bluebirds','#ccfc'],
    'Chelsea': ['#cfc','#chelseafc','#chelsea'],
    'Crystal Palace': ['#cpfc','#crystalpalacefc','#crystalpalace'],
    'Everton': ['#efc','#everton', '#evertonfc'],
    'Fulham': ['#ffc', '#fulhamfc'],
    'Huddersfield': ["#htafc", "#Huddersfieldfc"],
    'Hull': ['#hcafc','#hullcity'],
    'Leicester': ['#lcfc','#leicestercity', '#leicestercityfc'],
    'Liverpool': ['#lfc','#liverpool','#liverpoolfc'],
    'Manchester City': ['#mcfc','#mancity','#manchestercity'],
    'Manchester Utd': ['#mufc','#manutd','#manchesterunited'],
    'Newcastle': ['#nufc','#newcastlefc'],
    'Norwich': ['#ncfc','#norwichcity'],
    'QPR': ['#qpr', '#qprfc'],
    'Southampton': ['#saintsfc','#Southamptonfc'],
    'Stoke': ['#scfc','#stokecity'],
    'Sunderland': ['#sufc','#sunderland'],
    'Swansea': ['#swans','#swanseafc'],
    'Tottenham': ['#thfc','#tottenhamhotspur','#tottenham'],
    'Watford':['#watfordfc','watford'],
    'West Brom': ['#wbafc','#wba','#westbrom','#westbromwichalbion'],
    'West Ham': ['#whufc','#westham','#hammers'],
    'Wolves': ['#wolverhamptonfc','#wolvesfc','#wwfc','#wolverhamptonwanderers']
}

In [83]:
tweets = tweets[~tweets.tweet_hashtags.isna()].reset_index(drop=True) # alcuni hanno il tag unito tipo #ciao#tag#sbagliato

In [84]:
import numpy as np
from tqdm import tqdm
ind_to_remove = []
for ind,t,tags in tqdm(zip(tweets.index, tweets.team, tweets.tweet_hashtags)):
    tags = tags.split(' ')
    tags = [x.lower() for x in tags]
    x = [x for x in tags if x not in HASTAGS[t]]
    if x:
        ind_to_remove.append(ind)

155791it [00:00, 622338.43it/s]


In [85]:
tweets = tweets.drop(ind_to_remove)

In [86]:
tweets[['match_round','home_team','away_team']].drop_duplicates().shape

(380, 3)

In [87]:
tweets = tweets[['match_round','home_team','away_team','team','tweet_text']].reset_index(drop=True)

In [88]:
polarity = []
for tw in tqdm(tweets.tweet_text):
    pol = 0
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) (\w+:\/\/\S+)", " ", tw).split())
    if 'http' in tweet or 'https' in tweet:
        polarity.append(42)
    else:
        analysis = TextBlob(tweet)
        tp = analysis.sentiment.polarity
        if tp!=0:
            pol += analysis.sentiment.polarity
        polarity.append(pol)

100%|██████████████████████████████████████████████████████████████████████████| 75194/75194 [00:15<00:00, 4879.07it/s]


In [89]:
tweets['polarity'] = polarity

In [90]:
tweets.head(1000)

,match_round,home_team,away_team,team,tweet_text,polarity
0,38,Brighton,Manchester City,Brighton,Cannot decide who I love more between him and Calderón #bhafc,5.000000e-01
1,38,Brighton,Manchester City,Brighton,Do it for Bruno.. #BHAFC,0.000000e+00
2,38,Brighton,Manchester City,Brighton,"Everyone in the #BHAFC fam Let’s change our profile pics for the final game to the man,the legend,El Capitano Bruno Saltor We salute you sir 👏👏👏🔵⚪️",-2.000000e-01
3,38,Brighton,Manchester City,Brighton,Less of the unlikely. He’s got it covered 😉 #bhafc,-3.333333e-01
4,38,Brighton,Manchester City,Brighton,"Saw him for the 1st time in a pre season friendly at Lewes just after we signed him. Texted a mate saying, “We have a PLAYER!” It’s been an absolute pleasure Senor Bruno. Top player, top man. #BHAFC ⚽️ 🇪🇸",4.171875e-01
5,38,Brighton,Manchester City,Brighton,Duffy vows to show true colours for Albion against Man City https://www.theargus.co.uk/sport/17633106.everton-fan-duffy-determined-to-beat-city-even-if-it-helps-liverpool/?ref=twtrec #bhafc,4.200000e+01
6,38,Brighton,Manchester City,Brighton,Just when I thought Boutique couldn’t get any worse 🤣🤦🏻‍♂️ #BHAFC,-4.000000e-01
7,38,Brighton,Manchester City,Brighton,Should be a good show tomorrow morning at 11 on @1BTNradio. #bhafc,7.000000e-01
8,38,Brighton,Manchester City,Brighton,"Rather ironically, I spoke to the man who spoilt Bruno's #bhafc debut back in 2012 with two goals for Swindon. Liverpool fanatic Alan Navarro is hoping for a bit of help from his old club in The Argus tomorrow.",-6.666667e-02
9,38,Brighton,Manchester City,Brighton,Playoff for this game. #BHAFC⚪🔵⚪ 15 minutes. @BHAFCharlie @bhafcjames,-4.000000e-01


In [91]:
tweets[['match_round','home_team','away_team']].drop_duplicates().shape

(380, 3)

In [92]:
tweets = tweets[tweets.polarity!=42].sort_values(by=['match_round','home_team','away_team']).reset_index(drop=True)
tot = 0
pol = 0
pos = 0
neg = 0
mr,ht,aw,t= tweets.loc[0,'match_round'], tweets.loc[0,'home_team'],tweets.loc[0,'away_team'],tweets.loc[0,'team']
result = []
for r_mr,r_ht,r_aw,r_t,r_pol in tqdm(zip(tweets.match_round, tweets.home_team, tweets.away_team, tweets.team, tweets.polarity)):
    if r_mr==mr and r_ht==ht and r_aw==aw and r_t==t:
        if r_pol!=0:
            tot += 1
            pol += r_pol
            if r_pol > 0: 
                pos += 1
            else:
                neg += 1
    else:
        result.append([mr,ht,aw,t,pol/tot, pos/tot, neg/tot])
        mr,ht,aw,t = r_mr,r_ht,r_aw,r_t
        if r_pol!=0:
            tot=1
            pol=r_pol
            if r_pol > 0: 
                pos = 1
                neg = 0
            else:
                neg = 1
                pos = 1
        else:
            tot, pol, pos, neg = 0, 0, 0, 0
result.append([mr,ht,aw,t,pol/tot, pos/tot, neg/tot])

67301it [00:00, 1070608.25it/s]


In [93]:
len(result)

760

In [94]:
final_df = pd.DataFrame(columns=['match_round','home_team','away_team','team','sentiment_analysis'])

In [95]:
final_df.match_round = [x[0] for x in result]
final_df.home_team = [x[1] for x in result]
final_df.away_team = [x[2] for x in result]
final_df.team = [x[3] for x in result]
final_df.sentiment_analysis = [x[4] for x in result]

In [96]:
final_df.sort_values(by=['match_round','home_team']).reset_index(drop=True)

,match_round,home_team,away_team,team,sentiment_analysis
0,1,Arsenal,Manchester City,Arsenal,0.251956
1,1,Arsenal,Manchester City,Manchester City,0.343018
2,1,Bournemouth,Cardiff,Bournemouth,0.262046
3,1,Bournemouth,Cardiff,Cardiff,0.236831
4,1,Fulham,Crystal Palace,Fulham,0.278902
5,1,Fulham,Crystal Palace,Crystal Palace,0.335850
6,1,Huddersfield,Chelsea,Huddersfield,0.221228
7,1,Huddersfield,Chelsea,Chelsea,0.257366
8,1,Liverpool,West Ham,Liverpool,0.278667
9,1,Liverpool,West Ham,West Ham,0.389500


In [97]:
final_df.shape

(760, 5)

In [98]:
first = []
second = []
for i in range(final_df.shape[0] // 2):
    first.append(final_df.loc[i*2,'sentiment_analysis'])
    second.append(final_df.loc[i*2+1,'sentiment_analysis'])

In [99]:
final_df

,match_round,home_team,away_team,team,sentiment_analysis
0,1,Arsenal,Manchester City,Arsenal,0.251956
1,1,Arsenal,Manchester City,Manchester City,0.343018
2,1,Bournemouth,Cardiff,Bournemouth,0.262046
3,1,Bournemouth,Cardiff,Cardiff,0.236831
4,1,Fulham,Crystal Palace,Fulham,0.278902
5,1,Fulham,Crystal Palace,Crystal Palace,0.335850
6,1,Huddersfield,Chelsea,Huddersfield,0.221228
7,1,Huddersfield,Chelsea,Chelsea,0.257366
8,1,Liverpool,West Ham,Liverpool,0.278667
9,1,Liverpool,West Ham,West Ham,0.389500


In [100]:
feature = final_df[['match_round','home_team','away_team']].drop_duplicates().reset_index(drop=True)
feature['ht_sentiment'] = first
feature['at_sentiment'] = second
feature

,match_round,home_team,away_team,ht_sentiment,at_sentiment
0,1,Arsenal,Manchester City,0.251956,0.343018
1,1,Bournemouth,Cardiff,0.262046,0.236831
2,1,Fulham,Crystal Palace,0.278902,0.335850
3,1,Huddersfield,Chelsea,0.221228,0.257366
4,1,Liverpool,West Ham,0.278667,0.389500
5,1,Manchester Utd,Leicester,0.219423,0.217218
6,1,Newcastle,Tottenham,0.073739,0.161378
7,1,Southampton,Burnley,0.237583,0.289004
8,1,Watford,Brighton,0.093000,0.225737
9,1,Wolves,Everton,0.282895,0.188209


In [101]:
df = pd.read_csv('premier_lgb_ready.csv')

df = df[df.league_year==2018].reset_index(drop=True)

df

,10bet_1,10bet_2,10bet_X,10bet_opening_1,10bet_opening_2,10bet_opening_X,188bet_1,188bet_2,188bet_X,188bet_opening_1,...,home_team_rank,home_team_points,home_team_goal_difference,home_team_goal_fatti,home_team_goal_subiti,away_team_rank,away_team_points,away_team_goal_difference,away_team_goal_fatti,away_team_goal_subiti
0,1.56,7.00,3.80,1.40,8.50,4.45,1.55,6.80,4.00,1.32,...,0,0,0,0,0,0,0,0,0,0
1,4.75,1.87,3.40,4.40,1.83,3.60,4.40,1.88,3.55,4.05,...,0,0,0,0,0,0,0,0,0,0
2,1.87,4.35,3.65,1.83,4.65,3.50,1.86,4.35,3.65,2.03,...,0,0,0,0,0,0,0,0,0,0
3,2.55,2.80,3.40,2.30,3.20,3.30,2.56,2.82,3.35,2.36,...,0,0,0,0,0,0,0,0,0,0
4,6.70,1.57,3.85,6.15,1.54,4.10,6.60,1.57,3.90,6.00,...,0,0,0,0,0,0,0,0,0,0
5,2.50,3.15,3.05,2.25,3.30,3.25,2.51,3.05,3.15,2.25,...,0,0,0,0,0,0,0,0,0,0
6,2.40,3.25,3.15,2.50,2.90,3.30,2.38,3.20,3.25,2.50,...,0,0,0,0,0,0,0,0,0,0
7,1.24,12.00,6.80,1.33,9.75,5.00,1.22,12.50,6.90,1.28,...,0,0,0,0,0,0,0,0,0,0
8,2.00,4.40,3.20,1.95,4.10,3.45,2.05,4.20,3.20,1.98,...,0,0,0,0,0,0,0,0,0,0
9,4.25,1.80,3.95,3.85,1.91,3.80,4.25,1.81,3.95,3.30,...,0,0,0,0,0,0,0,0,0,0


In [102]:
f = pd.merge(df,feature,how='left', on =['match_round','home_team','away_team'])
f.head()

,10bet_1,10bet_2,10bet_X,10bet_opening_1,10bet_opening_2,10bet_opening_X,188bet_1,188bet_2,188bet_X,188bet_opening_1,...,home_team_goal_difference,home_team_goal_fatti,home_team_goal_subiti,away_team_rank,away_team_points,away_team_goal_difference,away_team_goal_fatti,away_team_goal_subiti,ht_sentiment,at_sentiment
0,1.56,7.00,3.80,1.40,8.50,4.45,1.55,6.80,4.00,1.32,...,0,0,0,0,0,0,0,0,0.219423,0.217218
1,4.75,1.87,3.40,4.40,1.83,3.60,4.40,1.88,3.55,4.05,...,0,0,0,0,0,0,0,0,0.073739,0.161378
2,1.87,4.35,3.65,1.83,4.65,3.50,1.86,4.35,3.65,2.03,...,0,0,0,0,0,0,0,0,0.262046,0.236831
3,2.55,2.80,3.40,2.30,3.20,3.30,2.56,2.82,3.35,2.36,...,0,0,0,0,0,0,0,0,0.278902,0.335850
4,6.70,1.57,3.85,6.15,1.54,4.10,6.60,1.57,3.90,6.00,...,0,0,0,0,0,0,0,0,0.221228,0.257366


In [103]:
# f=f[['label','match_date','home_team','away_team','home_goals','away_goals','ht_sentiment','at_sentiment']]

In [104]:
import lightgbm as lgb

from sklearn.model_selection import train_test_split

X = f.drop(['label','match_date','home_team','away_team','home_goals','away_goals'
           ],axis=1)
y = f.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=False)

In [105]:
model = lgb.LGBMClassifier(max_depth=4,learning_rate=0.001287, n_estimators=868, num_leaves=34,min_child_samples=29)
model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
correct = 0
for ind,true,predicted in zip(y_test.index, y_test, predicted_y):
    if true == predicted:
        correct+=1
correct*100 / y_test.shape[0]

57.89473684210526

In [66]:
# baseline che usa solo le quote
result = []
for p1,pX,p2 in zip(X_test['10bet_1'], X_test['10bet_X'], X_test['10bet_2']):
    if min([p1,pX,p2]) == p1: result.append(1)
    elif min([p1,pX,p2]) == pX: result.append(0)
    else: result.append(2)

correct = 0
for ind,true,predicted in zip(y_test.index, y_test, result):
    if true == predicted:
        correct+=1
correct*100 / y_test.shape[0]

53.94736842105263

In [106]:
only_sentiment = f[['ht_sentiment','at_sentiment','label']]
only_sentiment

,ht_sentiment,at_sentiment,label
0,0.219423,0.217218,1
1,0.073739,0.161378,2
2,0.262046,0.236831,1
3,0.278902,0.335850,2
4,0.221228,0.257366,2
5,0.093000,0.225737,1
6,0.282895,0.188209,0
7,0.278667,0.389500,1
8,0.237583,0.289004,0
9,0.251956,0.343018,2


In [111]:
pred = []
for h,a in zip(only_sentiment.ht_sentiment, only_sentiment.at_sentiment):
    if h-a> 0.005:
        pred.append(1)
    elif a-h>0.005:
        pred.append(2)
    else:
        pred.append(0)

In [112]:
correct = 0
for ind,true,predicted in zip(y_test.index, y_test, pred):
    if true == predicted:
        correct+=1
correct*100 / y_test.shape[0]

35.526315789473685